In [ ]:
import pandas as pd
import numpy as np
import nltk
df = pd.read_csv('train.csv')


In [ ]:
df = df[df['classes'].isin(["regular","figurative","irony","sarcasm"])]

df.describe()


,tweets,classes
count,9163,9163
unique,9163,1
top,Be aware dirty step to get money #staylight ...,figurative
freq,1,9163


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

def pre_process(temp, ngram_range=(1,1)):
    for i in range(len(temp)):
        # Remove hyperlinks
        temp = temp.apply(lambda text: re.sub(r'http\S+', '', text))
        # Remove twitter usernames
        temp = temp.apply(lambda text: re.sub(r'@\S+', '', text))
        # Remove special characters
        temp = temp.apply(lambda text: re.sub(r'[^\w\s]', '', text))
        # Remove whitespace
        temp = temp.apply(lambda text: text.strip())

    # Create CountVectorizer with specified ngram range
    vectorizer = CountVectorizer(stop_words='english', ngram_range=ngram_range)
    # Fit vectorizer on pre-processed data
    vectorizer.fit(temp)
    # Transform pre-processed data using the fitted vectorizer
    temp = vectorizer.transform(temp)
    return temp, vectorizer

In [ ]:
# Pre-process tweets with N-grams

y = df['classes']
X, vectorizer = pre_process(df['tweets'], ngram_range=(1,2))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TF-IDF transformer and fit_transform the count matrix
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train)

# Transform the test data using the same transformer
X_test_tfidf = tfidf_transformer.transform(X_test)

In [ ]:
import re
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')  # Or 'median' depending on your data
X_train_tfidf_imputed = imputer.fit_transform(X_train_tfidf)
X_test_tfidf_imputed = imputer.transform(X_test_tfidf)


Naives Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf1 = MultinomialNB()
clf1.fit(X_train_tfidf, y_train)
pred = clf1.predict(X_test_tfidf)
acc_score = metrics.accuracy_score(y_test, pred)
precision = metrics.precision_score(y_test, pred,average='weighted',zero_division=0)
recall = metrics.recall_score(y_test, pred,average='weighted',zero_division=0)
f1_score = metrics.f1_score(y_test, pred, average='weighted',zero_division=0)
print(acc_score)
print(precision)
print(recall)
print(f1_score)

ValueError: ignored

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
l1 = LogisticRegression(multi_class='multinomial', solver='lbfgs')
l1 = l1.fit(X_train_tfidf, y_train)
lrpred = l1.predict(X_test_tfidf)
acc_score = metrics.accuracy_score(y_test, lrpred)
precision = metrics.precision_score(y_test,lrpred,average='weighted',zero_division=0)
recall = metrics.recall_score(y_test,lrpred,average='weighted',zero_division=0)
f1_score = metrics.f1_score(y_test, lrpred, average='weighted',zero_division=0)
print(acc_score)
print(precision)
print(recall)
print(f1_score)


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1 = rf1.fit(X_train_tfidf, y_train)
rfpred = rf1.predict(X_test_tfidf)
acc_score = metrics.accuracy_score(y_test, rfpred)
precision = metrics.precision_score(y_test,rfpred,average='weighted',zero_division=0)
recall = metrics.recall_score(y_test,rfpred,average='weighted',zero_division=0)
f1_score = metrics.f1_score(y_test, rfpred, average='weighted',zero_division=0)
print(acc_score)
print(precision)
print(recall)
print(f1_score)

Perceptron

In [ ]:
from sklearn.linear_model import Perceptron
p1 = Perceptron()
p1 = p1.fit(X_train_tfidf,y_train)
ppred = p1.predict(X_test_tfidf)
acc_score = metrics.accuracy_score(y_test, ppred)
precision = metrics.precision_score(y_test,ppred,average='weighted')
recall = metrics.recall_score(y_test,ppred,average='weighted')
f1_score = metrics.f1_score(y_test, ppred, average='weighted')
print(acc_score)
print(precision)
print(recall)
print(f1_score)


In [ ]:
import pickle

# train the Naive Bayes model
model = clf1
#print(X_train_tfidf)


# save the model as a pickle file
with open('nb_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# train the Linear Regression model
model = l1
#print(X_train_tfidf)


# save the model as a pickle file
with open('lg_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# train the Random Forest model
model = rf1
#print(X_train_tfidf)


# save the model as a pickle file
with open('rf_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# train the Perceptron model
model = p1
#print(X_train_tfidf)


# save the model as a pickle file
with open('p_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [ ]:
import pandas as pd
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer

# Load the preprocessed data
df = pd.read_csv('train.csv')

# Load the trained naive bayes model
with open('nb_model.pkl', 'rb') as f:
    nb = pickle.load(f)

    # Load the logistic regression model
with open('nb_model.pkl', 'rb') as f:
    lg = pickle.load(f)

    # Load the random forest model
with open('nb_model.pkl', 'rb') as f:
    rf = pickle.load(f)

    # Load the perceptron model
with open('nb_model.pkl', 'rb') as f:
    p = pickle.load(f)

In [ ]:
def predict_sentiment(text, model, vectorizer):
    # Pre-process input text
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    # Vectorize input text using the same vectorizer
    X = vectorizer.transform([text])

    # Make prediction using the model
    pred = model.predict(X)
    return pred

In [ ]:
while(True):
  text = input("Enter a tweet to predict its sentiment: ")
  if text == "":
      break
  else:
      # Make prediction on the preprocessed tweet using the model and vectorizer trained on the training data
      pred = predict_sentiment(text, clf1, vectorizer)
      print(f"Predicted sentiment for Naive Bayes: {pred}")

      # Make prediction on the preprocessed tweet using the model and vectorizer trained on the training data
      pred = predict_sentiment(text, l1, vectorizer)
      print(f"Predicted sentiment for Logistic Regression: {pred}")

      # Make prediction on the preprocessed tweet using the model and vectorizer trained on the training data
      pred = predict_sentiment(text, rf1, vectorizer)
      print(f"Predicted sentiment for Random Forest: {pred}")

      # Make prediction on the preprocessed tweet using the model and vectorizer trained on the training data
      pred = predict_sentiment(text, p1, vectorizer)
      print(f"Predicted sentiment for Perceptron: {pred}")